In [25]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']
API_URL = os.environ['HF_API_NER_BASE']

In [26]:
# Helper function
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None,ENDPOINT_URL=os.environ['HF_API_SUMMARY_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

In [27]:
import gradio as gr

In [ ]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        label = token["entity"]
        word = token["word"]

        clean_word = word.replace("##", "")

        if merged_tokens:
            prev = merged_tokens[-1]
            prev_label = prev["entity"]

            if label.endswith(prev_label.split("-")[-1]) and (label.startswith("I-") or label.startswith("B-")):
                prev["word"] += clean_word
                prev["end"] = token["end"]
                prev["score"] = (prev["score"] + token["score"]) / 2
                continue

            if word.startswith("##") and prev_label.startswith("B-"):
                prev["word"] += clean_word
                prev["end"] = token["end"]
                prev["score"] = (prev["score"] + token["score"]) / 2
                continue
            
            if prev["entity"].endswith("PER") and len(prev["word"]) == 1 and word.startswith("##"):
                prev["word"] += clean_word
                prev["end"] = token["end"]
                prev["score"] = (prev["score"] + token["score"]) / 2
                continue


        merged_tokens.append({
            "entity": label,
            "word": clean_word,
            "start": token["start"],
            "end": token["end"],
            "score": token["score"]
        })

    return merged_tokens


def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(
    fn=ner,
    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
    outputs=[gr.HighlightedText(label="Text with entities")],
    title="NER with Fine-Tuned BART ",
    description="Highlights people, organizations, and locations in your text.",
    allow_flagging="never",
    examples=[
        ["My name is Haarish, I'm building DeepLearningAI and I live in Chennai"],
        ["Elon Musk founded SpaceX in the United States"]
    ]
)

demo.launch(share=True, server_port=int(os.environ.get("PORT4", 7860)))

Closing server running on port: 7863
Closing server running on port: 7863
Closing server running on port: 7863
Closing server running on port: 7863
Closing server running on port: 7863
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7863
Running on local URL:  https://0.0.0.0:7863
IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Missing file: /usr/local/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.9/site-packages/gradio
